# SIMCCT User Analytics

This Jupyter Notebook is for doing some quick and dirty tests for running data 
analytics operations on the MongoDB and Redis data persistence. The purpose of 
which is to then put into a pipeline and then create endpoints for each type 
of query and analysis that can be done.

In [1]:
# Plotly imports

import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

chart_studio.tools.set_credentials_file(
    username='codeninja55', 
    api_key='mLp691cLJDdKaNgJykR4'
)

chart_studio.tools.set_config_file(
    world_readable=True,
    sharing='public'
)

In [7]:
# imports
import datetime
from os import environ as env
from pymongo import MongoClient
import pandas as pd
from redis import Redis

In [3]:
conn = MongoClient(env.get('MONGO_URI'))

db_name = 'arc_dev'
collection = 'users'
db = conn[db_name]

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'arc_dev')

In [5]:
cursor = db[collection].find(
    {
        'profile': {'$exists': True}
    }, 
    projection={'password': 0, '_id': False}
)
df = pd.DataFrame(list(cursor))
df.head()

,email,first_name,last_name,profile,admin_profile,saved_alloys,active,admin,disable_admin,verified,created,last_updated,ratings,login_data,last_login
0,ironman@avengers.io,Tony,Stark,"{'aim': 'Experimentation', 'highest_education'...",{'position': 'Genius Playboy Billionaire Phila...,"[{'_id': 5d985c0b181db01c07f14633, 'name': 'Ua...",True,True,False,False,2019-10-05 09:02:03.539,2019-10-05 09:02:44.960,"[{'rating': 1, 'created_date': 2019-09-25 09:0...",[{'created_datetime': 2019-09-25 09:02:03.6720...,2019-10-05 09:02:44.925
1,black_widow@marvel.io,Natasha,Romanoff,"{'aim': 'Engineering Work', 'highest_education...","{'position': 'Russian Superagent', 'mobile_num...","[{'_id': 5d985c0b181db01c07f1464e, 'name': 'Th...",True,True,False,False,2019-10-05 09:02:03.539,2019-10-05 09:02:03.539,"[{'rating': 3, 'created_date': 2019-09-25 09:0...",[{'created_datetime': 2019-09-25 09:02:03.6720...,NaT
2,captain_america@avengers.io,Steve,Rogers,"{'aim': 'Experimentation', 'highest_education'...","{'position': 'Captain', 'mobile_number': '+614...","[{'_id': 5d985c0b181db01c07f1466a, 'name': 'Ar...",True,True,False,False,2019-10-05 09:02:03.539,2019-10-05 09:02:03.539,"[{'rating': 3, 'created_date': 2019-09-25 09:0...",[{'created_datetime': 2019-09-25 09:02:03.6720...,NaT
3,hulk@avengers.io,Bruce,Banner,"{'aim': 'Research', 'highest_education': 'High...","{'position': 'Angry Giant cum cool guy now', '...","[{'_id': 5d985c0c181db01c07f14674, 'name': 'al...",True,True,False,False,2019-10-05 09:02:03.539,2019-10-05 09:02:03.539,"[{'rating': 4, 'created_date': 2019-09-25 09:0...",[{'created_datetime': 2019-09-25 09:02:03.6720...,NaT
4,hawkeye@avengers.io,Clint,Barton,"{'aim': 'Engineering Work', 'highest_education...","{'position': 'Marksmen', 'mobile_number': '+61...","[{'_id': 5d985c0c181db01c07f1467d, 'name': 'Ad...",True,True,False,False,2019-10-05 09:02:03.539,2019-10-05 09:02:03.539,"[{'rating': 2, 'created_date': 2019-09-25 09:0...",[{'created_datetime': 2019-09-25 09:02:03.6720...,NaT


## User Profile Data

In [ ]:
pipeline = [
    {'$unwind': '$profile'},
    {'$project': {'profile': 1, '_id': False}},
]

res = db[collection].aggregate(pipeline)
list(res)

In [ ]:
pipeline = [
    {'$unwind': '$profile'},
    {'$project': {
        'aim': '$profile.aim', 
        'highest_education': '$profile.highest_education', 
        'sci_tech_exp': '$profile.sci_tech_exp', 
        'phase_transform_exp': '$profile.phase_transform_exp',
        '_id': 0
        }
    },
]

res = db[collection].aggregate(pipeline)
profile_df = pd.DataFrame(list(res))
profile_df

In [ ]:
list(profile_df['aim'].unique())

In [ ]:
list(profile_df['aim'].value_counts())

In [ ]:
# layout = go.Layout(
#     title='User Profile Aim',
#     xaxis=dict(title='User Aims'),
#     yaxis=dict(title='Count')
# )

# fig = go.Figure(layout=layout)
fig = make_subplots(
    rows=2, 
    cols=2,
    subplot_titles=[
        'Aim', 
        'Highest Education', 
        'Science Tech. Experience',
        'Phase Transform Experience'
    ]
)

trace_aim = go.Bar(x=list(profile_df['aim'].unique()), y=list(profile_df['aim'].value_counts()))
trace_edu = go.Bar(x=list(profile_df['highest_education'].unique()), y=list(profile_df['highest_education'].value_counts()))
trace_sci = go.Bar(x=list(profile_df['sci_tech_exp'].unique()), y=list(profile_df['sci_tech_exp'].value_counts()))
trace_pha = go.Bar(x=list(profile_df['phase_transform_exp'].unique()), y=list(profile_df['phase_transform_exp'].value_counts()))

fig.add_trace(trace_aim, row=1, col=1)
fig.add_trace(trace_edu, row=1, col=2)
fig.add_trace(trace_sci, row=2, col=1)
fig.add_trace(trace_pha, row=2, col=2)

fig.update_layout(
    # height=800,
    # width=1200,
    showlegend=False,
    title_text="User Profile Answers"
)

py.iplot(fig, filename='user_profile_bar')
# pio.write_image(fig, file='user_profile_aim.png')

## Live Login Data

In [ ]:
cursor = db[collection].find(
    {
        'last_login': {'$exists': True}
    }, 
    projection={'password': 0, '_id': False}
)
df = pd.DataFrame(list(cursor))
df.head()

In [ ]:
pipeline = [
  {'$unwind': '$login_data'},
  {'$project': {'_id': 0, 'login_data': 1, 'email': 1}},
  {'$sort': {'login_data.created_datetime': 1}}
]

res = db[collection].aggregate(pipeline)
# login_df = pd.DataFrame(list(res))
list(res)

In [ ]:
pipeline = [
  {'$unwind': '$login_data'},
  {'$project': {
      '_id': 0, 
      'created_datetime': '$login_data.created_datetime', 
    }
  },
]

res = db[collection].aggregate(pipeline)
list(res)

In [ ]:
# Using graph_objects
import plotly.graph_objects as go

import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

df['Date'].head(n=10)

In [ ]:
df['AAPL.High'].head(n=10)

In [ ]:
fig = go.Figure([go.Scatter(x=df['Date'], y=df['AAPL.High'])])
fig.show()

In [ ]:
import plotly.graph_objects as go
import datetime

x = [datetime.datetime(year=2013, month=10, day=4),
     datetime.datetime(year=2013, month=11, day=5),
     datetime.datetime(year=2013, month=12, day=6)]

fig = go.Figure(data=[go.Scatter(x=x, y=[1, 3, 6])])
# Use datetime objects to set xaxis range
fig.update_layout(xaxis_range=[datetime.datetime(2013, 10, 17),
                               datetime.datetime(2013, 11, 20)])
fig.show()

In [ ]:
pipeline = [
  {'$unwind': '$login_data'},
  {'$project': {
      '_id': 0, 
      'timestamp': '$login_data.created_datetime', 
      'user': '$email',
    }
  },
]

res = db[collection].aggregate(pipeline)


# dt_idx = pd.to_datetime()

df = pd.DataFrame(list(res))

df['timestamp'] = pd.to_datetime(df['timestamp'])
# df.set_index('timestamp', inplace=True)

df = df.groupby(pd.Grouper(key='timestamp', freq='1min')).count().dropna()

# df = df.groupby(pd.Grouper(key='timestamp', freq='60s'))
# res = (pd.DataFrame(df.index[1:]) - pd.DataFrame(df.index[:-1]))
# df = df.to_frame().reset_index()
# df.resample('T').count()

# res['timestamp'].value_counts()
df

In [ ]:
fig = go.Figure()

trace = go.Scatter(x=df.index, y=df['user'])

fig.add_trace(trace)

fig.update_layout(
  showlegend=False,
  title_text="Logged in Users",
  xaxis_range=[
    datetime.datetime(2019, 10, 4),
    datetime.datetime(2019, 10, 5)
  ],
  xaxis_rangeslider_visible=True
)

py.iplot(fig, filename='user_login_timestamps')

## Logged In User Map

In [12]:
redis_uri = env.get('REDIS_URI')
client = Redis(redis_uri)
client

Redis<ConnectionPool<Connection<host=None,port=6379,db=0>>>

In [13]:
keys = client.keys(pattern=u'session*')
keys

[b'session:5d985c0b181db01c07f1464d.13a831ef8df04ab48b6dc2b0fbc58337']

In [23]:
import json 

for byte_key in keys:
  key = byte_key.decode('utf-8')
  print()
  sess_store = json.loads(client.get(key))
  print(sess_store)


None
